In [1]:
%matplotlib inline
import collections
import math
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
import tensorflow as tf

seed = 54321

%env TF_FORCE_GPU_ALLOW_GROWTH=true

env: TF_FORCE_GPU_ALLOW_GROWTH=true


In [2]:
url = 'http://cogcomp.org/Data/QA/QC/'
dir_name = 'data'

def download_data(dir_name, filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
  
    os.makedirs(dir_name, exist_ok=True)
    if not os.path.exists(os.path.join(dir_name,filename)):
        filepath, _ = urlretrieve(url + filename, os.path.join(dir_name,filename))
    else:
        filepath = os.path.join(dir_name, filename)
    
    statinfo = os.stat(filepath)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filepath)
    else:
        print(statinfo.st_size)
        raise Exception(
          'Failed to verify ' + filepath + '. Can you get to it with a browser?')
        
    return filepath

train_filename = download_data(dir_name, 'train_5500.label', 335858)
test_filename = download_data(dir_name, 'TREC_10.label',23354)

Found and verified data\train_5500.label
Found and verified data\TREC_10.label


In [3]:
def read_data(filename):
    '''
    Read data from a file with given filename
    Returns a list of strings where each string is a lower case word
    '''

    questions, categories, sub_categories = [], [], []     
    
    with open(filename,'r',encoding='latin-1') as f:        
        # Read each line
        for row in f:   
            row_str = row.split(":")        
            cat, sub_cat_and_question = row_str[0], row_str[1]
            tokens = sub_cat_and_question.split(' ')
            sub_cat, question = tokens[0], ' '.join(tokens[1:])        
            
            questions.append(question.lower().strip())
            categories.append(cat)
            sub_categories.append(sub_cat)
            

    return questions, categories, sub_categories

train_questions, train_categories, train_sub_categories = read_data(train_filename)
test_questions, test_categories, test_sub_categories = read_data(test_filename)

n_samples = 10
print(f"train_questions has {len(train_questions)} questions / {len(train_categories)} labels")
print("Some samples")
for question, cat, sub_cat in zip(train_questions[:n_samples], train_categories[:n_samples], train_sub_categories[:n_samples]):    
    print(f"\t{question} / cat - {cat} / sub_cat - {sub_cat}")
          
print(f"\ntest_questions has {len(test_questions)} questions / {len(test_categories)} labels")
print("Some samples")
for question, cat, sub_cat in zip(test_questions[:n_samples], test_categories[:n_samples], test_sub_categories[:n_samples]):    
    print(f"\t{question} / cat - {cat} / sub_cat - {sub_cat}")

train_questions has 5452 questions / 5452 labels
Some samples
	how did serfdom develop in and then leave russia ? / cat - DESC / sub_cat - manner
	what films featured the character popeye doyle ? / cat - ENTY / sub_cat - cremat
	how can i find a list of celebrities ' real names ? / cat - DESC / sub_cat - manner
	what fowl grabs the spotlight after the chinese year of the monkey ? / cat - ENTY / sub_cat - animal
	what is the full form of .com ? / cat - ABBR / sub_cat - exp
	what contemptible scoundrel stole the cork from my lunch ? / cat - HUM / sub_cat - ind
	what team did baseball 's st. louis browns become ? / cat - HUM / sub_cat - gr
	what is the oldest profession ? / cat - HUM / sub_cat - title
	what are liver enzymes ? / cat - DESC / sub_cat - def
	name the scar-faced bounty hunter of the old west . / cat - HUM / sub_cat - ind

test_questions has 500 questions / 500 labels
Some samples
	how far is it from denver to aspen ? / cat - NUM / sub_cat - dist
	what county is modesto , cal

In [4]:
# Define training and testing
train_df = pd.DataFrame(
    {'question': train_questions, 'category': train_categories, 'sub_category': train_sub_categories}
)
test_df = pd.DataFrame(
    {'question': test_questions, 'category': test_categories, 'sub_category': test_sub_categories}
)

train_df.head(n=10)

,question,category,sub_category
0,how did serfdom develop in and then leave russ...,DESC,manner
1,what films featured the character popeye doyle ?,ENTY,cremat
2,how can i find a list of celebrities ' real na...,DESC,manner
3,what fowl grabs the spotlight after the chines...,ENTY,animal
4,what is the full form of .com ?,ABBR,exp
5,what contemptible scoundrel stole the cork fro...,HUM,ind
6,what team did baseball 's st. louis browns bec...,HUM,gr
7,what is the oldest profession ?,HUM,title
8,what are liver enzymes ?,DESC,def
9,name the scar-faced bounty hunter of the old w...,HUM,ind


In [5]:
# Shuffle the data for better randomization
train_df = train_df.sample(frac=1.0, random_state=seed)

In [6]:
train_df

,question,category,sub_category
5267,what is an aurora ?,DESC,def
21,what articles of clothing are tokens in monopo...,ENTY,other
3258,what causes rust ?,DESC,reason
1356,what does an irate car owner call iron oxide ?,ENTY,termeq
1529,what do we call the imaginary line along the t...,LOC,other
...,...,...,...
1174,who did bobby fischer beat to win the world ch...,HUM,ind
2020,what poet wrote,HUM,ind
4192,"in order from the top , the four stripes on a ...",ENTY,color
4746,what is the definition of cecum ?,DESC,def


In [7]:
# Generate the label to ID mapping
unique_cats = train_df["category"].unique()
labels_map = dict(zip(unique_cats, np.arange(unique_cats.shape[0])))
print(f"Label->ID mapping: {labels_map}")

n_classes = len(labels_map)

# Convert all string labels to IDs
train_df["category"] = train_df["category"].map(labels_map)
test_df["category"] = test_df["category"].map(labels_map)

# View some data
train_df.head(n=10)

Label->ID mapping: {'DESC': 0, 'ENTY': 1, 'LOC': 2, 'NUM': 3, 'HUM': 4, 'ABBR': 5}


,question,category,sub_category
5267,what is an aurora ?,0,def
21,what articles of clothing are tokens in monopo...,1,other
3258,what causes rust ?,0,reason
1356,what does an irate car owner call iron oxide ?,1,termeq
1529,what do we call the imaginary line along the t...,2,other
3631,why is hockey so violent ?,0,reason
4802,how many characters makes up a word for typing...,3,count
2288,what peter blatty novel recounts the horrors o...,1,cremat
803,what is measured in curies ?,0,def
4472,what does seccession mean ?,0,def


In [8]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(train_df, test_size=0.1)
print(f"Train size: {train_df.shape}")
print(f"Valid size: {valid_df.shape}")

# Print data
train_df.head()

Train size: (4906, 3)
Valid size: (546, 3)


,question,category,sub_category
4144,where is the official `` zero '' of the sea le...,2,other
5036,what is the name of the song that dracula play...,1,cremat
2969,"who wrote the book , `` song of solomon '' ?",4,ind
349,how do doctors diagnose bone cancer ?,0,manner
870,which killer whale died at sea world of a fung...,1,animal


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Define a tokenizer and fit on train data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df["question"].tolist())

# Derive the vocabulary size
n_vocab = len(tokenizer.index_word) + 1
print(f"Vocabluary size: {n_vocab}")

Vocabluary size: 7874


In [10]:
# Split each string by " ", compute length of the list, get the percentiles
train_df["question"].str.split(" ").str.len().describe(percentiles=[0.01, 0.5, 0.99])

count    4906.000000
mean       10.044435
std         3.764644
min         2.000000
1%          4.000000
50%         9.000000
99%        21.000000
max        37.000000
Name: question, dtype: float64

In [11]:
# Convert each list of tokens to a list of IDs, using tokenizer's mapping
train_sequences = tokenizer.texts_to_sequences(train_df["question"].tolist())
train_labels = train_df["category"].values
valid_sequences = tokenizer.texts_to_sequences(valid_df["question"].tolist())
valid_labels = valid_df["category"].values
test_sequences = tokenizer.texts_to_sequences(test_df["question"].tolist())
test_labels = test_df["category"].values

max_seq_length = 22

# Pad shorter sentences and truncate longer ones (maximum length: max_seq_length)
preprocessed_train_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    train_sequences, maxlen=max_seq_length, padding='post', truncating='post'
)
preprocessed_valid_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    valid_sequences, maxlen=max_seq_length, padding='post', truncating='post'
)
preprocessed_test_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    test_sequences, maxlen=max_seq_length, padding='post', truncating='post'
)

In [12]:
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
import tensorflow.keras.regularizers as regularizers
from tensorflow.keras.models import Model

K.clear_session()

# Input layer takes word IDs as inputs
word_id_inputs = layers.Input(shape=(max_seq_length,), dtype='int32')

# Get the embeddings of the inputs / out [batch_size, sent_length, output_dim]
embedding_out = layers.Embedding(input_dim=n_vocab, output_dim=64)(word_id_inputs)


# For all layers: in [batch_size, sent_length, emb_size] / out [batch_size, sent_length, 100]
conv1_1 = layers.Conv1D(
    100, kernel_size=3, strides=1, padding='same', activation='relu'
)(embedding_out)
conv1_2 = layers.Conv1D(
    100, kernel_size=4, strides=1, padding='same', activation='relu'
)(embedding_out)
conv1_3 = layers.Conv1D(
    100, kernel_size=5, strides=1, padding='same', activation='relu'
)(embedding_out)

# in previous conve outputs / out [batch_size, sent_length, 300]
conv_out = layers.Concatenate(axis=-1)([conv1_1, conv1_2, conv1_3])

pool_over_time_out = layers.MaxPool1D(pool_size=max_seq_length, padding='valid')(conv_out)

# Flatten the unit length dimension
flatten_out = layers.Flatten()(pool_over_time_out)

# Compute the final output
out = layers.Dense(
    n_classes, activation='softmax',
    kernel_regularizer=regularizers.l2(0.001)
)(flatten_out)

# Define the model
cnn_model = Model(inputs=word_id_inputs, outputs=out)

# Compile the model with loss/optimzier/metrics
cnn_model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

cnn_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 22)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 22, 64)    │    503,936 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 22, 100)   │     19,300 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 22, 100)   │     25,700 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 22, 100)   │     32,100 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 22, 300)   │          0 │ conv1d[0][0],     │
│ (Concatenate)       │                   │            │ conv1d_1[0][0],   │
│                     │                   │            │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d       │ (None, 1, 300)    │          0 │ concatenate[0][0] │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 300)       │          0 │ max_pooling1d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 6)         │      1,806 │ flatten[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 582,842 (2.22 MB)

 Trainable params: 582,842 (2.22 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Call backs
lr_reduce_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=3, verbose=1,
    mode='auto', min_delta=0.0001, min_lr=0.000001
)

# Train the model
cnn_model.fit(
    preprocessed_train_sequences, train_labels, 
    validation_data=(preprocessed_valid_sequences, valid_labels),
    batch_size=128, 
    epochs=25,
    callbacks=[lr_reduce_callback]
)

Epoch 1/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.3053 - loss: 1.6922 - val_accuracy: 0.5769 - val_loss: 1.2306 - learning_rate: 0.0010
Epoch 2/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6546 - loss: 1.0790 - val_accuracy: 0.7179 - val_loss: 0.7977 - learning_rate: 0.0010
Epoch 3/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8015 - loss: 0.6510 - val_accuracy: 0.7949 - val_loss: 0.5911 - learning_rate: 0.0010
Epoch 4/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9113 - loss: 0.3692 - val_accuracy: 0.8516 - val_loss: 0.4734 - learning_rate: 0.0010
Epoch 5/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9604 - loss: 0.2088 - val_accuracy: 0.8700 - val_loss: 0.4190 - learning_rate: 0.0010
Epoch 6/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9821 - loss: 0.1299 - val_accuracy: 0.8755 - val_loss: 0.4054 - learning_rate: 0.0010
Epoch 7/25
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9936 - loss: 0.0831 - val_acc

In [14]:
cnn_model.evaluate(preprocessed_test_sequences, test_labels, return_dict=True)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8992 - loss: 0.3500 


{'accuracy': 0.8939999938011169, 'loss': 0.3707362115383148}